In [52]:
import twstock
import pandas as pd

# List of Taiwan ETF codes with corresponding names
etf_list = {
    '0050': '元大台灣50',
    '0051': '元大中型100',
    '0052': '富邦科技',
    # Add more ETF codes and names as needed
}

# Initialize an empty DataFrame to store data
df_all_etfs = pd.DataFrame()

# Loop through each ETF code and fetch data
for etf_code, etf_name in etf_list.items():
    stock = twstock.Stock(etf_code)
    target_price = stock.fetch_from(2024, 2)  # Fetch data from May 2020 onwards

    # Define column names
    name_attribute = [
        'Date', 'Capacity', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change','Transaction'
    ]

    # Create DataFrame for the current ETF
    df_etf = pd.DataFrame(columns=name_attribute, data=target_price)

    # Add a column for ETF code
    df_etf['ETF_Code'] = etf_code

    # Add a column for ETF name
    df_etf['ETF_Name'] = etf_name

    # Write the current ETF's DataFrame to a CSV file
    filename = f'/content/drive/MyDrive/Colab Notebooks/stock_test/etf_{etf_code}.csv'
    df_etf.to_csv(filename)

    # Concatenate the current ETF DataFrame with the overall DataFrame
    df_all_etfs = pd.concat([df_all_etfs, df_etf], ignore_index=True)

print(df_all_etfs)



         Date  Capacity    Turnover    Open    High     Low   Close  Change  \
0  2024-02-01   8812920  1180328029  134.35  134.50  133.60  134.35    0.00   
1  2024-02-02   9178958  1238629170  135.00  135.20  134.65  135.10    0.75   
2  2024-02-05  17961425  2439229519  135.10  136.20  135.00  135.95    0.85   
3  2024-02-15  44444918  6313384766  142.20  142.70  141.40  142.35    6.40   
4  2024-02-16  18273199  2589642250  142.35  142.35  141.20  141.30   -1.05   
..        ...       ...         ...     ...     ...     ...     ...     ...   
82 2024-03-18    622034    96789869  155.20  156.95  155.00  156.95    1.70   
83 2024-03-19    291168    45476280  156.10  156.60  155.35  156.40   -0.55   
84 2024-03-20    795381   124557552  156.60  157.65  155.00  155.85   -0.55   
85 2024-03-21    958730   152488497  157.85  160.00  157.85  160.00    4.15   
86 2024-03-22    415163    66263546  160.60  160.60  158.40  160.15    0.15   

    Transaction ETF_Code ETF_Name  
0          9600

In [56]:
import datetime
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

# 產生近7個實際日期
todayDate = datetime.datetime.now()
dateList = []
for i in range(7):
    iDate = todayDate - datetime.timedelta(days=i)
    dateList.append(iDate.strftime('%Y%m%d'))

# 建立儲存表
stockPriceData = pd.DataFrame()

# 迴圈日期下載資料
for iDate in dateList:

    # 下載證交所資料
    # 取得目標日期資料
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date=' + iDate + '&type=ALLBUT0999'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 判斷是否有空資料存在 若存在則跳離此次迴圈
    if ('很抱歉，沒有符合條件的資料!' in soup.text):
        continue

    # 整理證交所每日收盤行情表
    table = soup.find_all('table')[8]
    columnNames = table.find('thead').find_all('tr')[2].find_all('td')
    columnNames = [elem.getText() for elem in columnNames]
    rowDatas = table.find('tbody').find_all('tr')
    rows = list()
    for row in rowDatas:
        rows.append([elem.getText().replace(',', '').replace('--', '') for elem in row.find_all('td')])
    df = pd.DataFrame(data=rows, columns=columnNames)
    df = df[['證券代號', '證券名稱', '開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額']]
    df = df.rename({'證券代號': 'code', '證券名稱': 'name', '開盤價': 'open', '最高價': 'high',
                    '最低價': 'low', '收盤價': 'close', '成交股數': 'volume', '成交金額': 'value'}, axis=1)
    df.insert(0, 'date', iDate, True)
    df.insert(1, 'mkt', 'tse', True)

    # 儲存證交所資料
    stockPriceData = pd.concat([stockPriceData, df])
    time.sleep(1)

filename = '/content/drive/MyDrive/Colab Notebooks/stock_test/etf.csv'
stockPriceData.to_csv(filename)

# 呈現結果
print(stockPriceData)

          date  mkt  code      name    open    high     low   close   volume  \
0     20240322  tse  0050    元大台灣50  157.50  158.25  156.50  157.20  7163593   
1     20240322  tse  0051   元大中型100   78.65   78.80   78.30   78.40    44071   
2     20240322  tse  0052      富邦科技  160.60  160.60  158.40  160.15   415163   
3     20240322  tse  0053      元大電子   85.80   86.10   85.10   85.90    13454   
4     20240322  tse  0055  元大MSCI金融   25.31   25.45   25.12   25.30   886594   
...        ...  ...   ...       ...     ...     ...     ...     ...      ...   
1225  20240318  tse  9944        新麗   20.00   20.10   19.90   20.10   117734   
1226  20240318  tse  9945       潤泰新   35.00   35.15   34.60   34.70  7314745   
1227  20240318  tse  9946      三發地產   23.05   23.45   22.90   23.25   372887   
1228  20240318  tse  9955        佳龍   23.75   23.75   23.20   23.45   290487   
1229  20240318  tse  9958       世紀鋼  225.00  227.50  220.00  224.00  4086510   

           value  
0     1126524179  
1